### Imports

In [1]:
from keras.models import Model
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import CSVLogger, Callback
from datetime import datetime
import keras.backend as K
import extras.ourUtils as utils
import numpy as np
import Models
import sys
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import KFold
from extras.ourUtils import reverse_data_split, create_data_split

C:\Users\admin\AppData\Local\conda\conda\envs\tensorflow\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


### Init

In [2]:
batch_size = 64
nrEpochs = 10
full_train = True
path = 'C:/Users/DANN_maces_jaskmo/Documents/'
source_data = path + 'newTaperImg/Physionet/'
target_data = path + 'newTaperImg/Hospital/'
stdout_cell = sys.stdout
MIQ = ['DA', 'target', 'source']
kf = KFold(n_splits = 10)
n_subjects_phys = np.arange(1,21);
n_subjects_hosp = np.arange(1,35);

class FlipControle(Callback):
    def __init__(self, alphaIn):
        self.alpha = alphaIn
        print(K.get_value(lamFunk))
        
    def on_epoch_end(self, epoch, logs={}):
        p = (epoch+1)/nrEpochs
        K.set_value(self.alpha, (2/(1+np.exp(-10*p)))-1)
        print(K.get_value(lamFunk))


## Get data as generators

In [3]:
datagen = ImageDataGenerator(rescale=1./255)

# make a data generator for dplInput
def train_gen_DAnet(source, target, batch_size):
    half = batch_size//2
    while True:
        source_data, source_lable = source.next()
        target_data, target_lable = target.next()
        if len(source_lable) != batch_size or len(target_lable) != batch_size:
            continue
        dpl_data = np.concatenate((source_data[:half,...],target_data[:half,...]),axis=0)
               
        domain_tmp = np.ones(batch_size, dtype='int8')
        domain_tmp[half:] = domain_tmp[half:] * 0
        dpl_lable = np.concatenate((domain_tmp.reshape(batch_size,1),
                                       np.flip(domain_tmp,0).reshape(batch_size,1)),1)

        yield({'lplInput':source_data,'dplInput':dpl_data}, {'lplOut':source_lable,'dplOut':dpl_lable})
        
def test_gen_DAnet(source, target, batch_size):
    half = batch_size//2
    while True:
        source_data, source_lable = source.next()
        target_data, target_lable = target.next()
        if len(source_lable) != batch_size or len(target_lable) != batch_size:
            continue
        dpl_data = np.concatenate((source_data[:half,...],target_data[:half,...]),axis=0)
               
        domain_tmp = np.ones(batch_size, dtype='int8')
        domain_tmp[half:] = domain_tmp[half:] * 0
        dpl_lable = np.concatenate((domain_tmp.reshape(batch_size,1),
                                       np.flip(domain_tmp,0).reshape(batch_size,1)),1)

        yield({'lplInput':target_data,'dplInput':dpl_data}, {'lplOut':target_lable,'dplOut':dpl_lable})

In [4]:
whitelist = dir()
whitelist.append('whitelist')
whitelist.append('phys_split')
whitelist.append('hosp_split')
whitelist.append('this')

In [5]:
for phys_split, hosp_split in zip(kf.split(n_subjects_phys),kf.split(n_subjects_hosp)):

    # memory management
    this = sys.modules[__name__]
    for n in dir():
        if n not in whitelist: delattr(this, n)
    K.clear_session()

    #Define current time
    now = datetime.now()
    #print("PHYS:\n" + "Train:" + str(phys_split[0]) + '\n' + 'Test:' + str(phys_split[1]) + "\n")
    #print("Hosp:\n" + "Train:" + str(hosp_split[0]) + '\n' + 'Test:' + str(hosp_split[1]) + "\n")
    
    reverse_data_split(source_data)
    reverse_data_split(target_data)
    
    #Data split for physionet
    create_data_split(source_data,phys_split[1]+1)
    #Data split for target
    create_data_split(target_data,hosp_split[1]+1)
    
    
    #### Train data


    train_gen_source = datagen.flow_from_directory(source_data + '/train', target_size=(224, 224), 
                                                   batch_size=batch_size, class_mode='categorical', shuffle=True)

    train_gen_target = datagen.flow_from_directory(target_data + '/train', target_size=(224, 224), 
                                                   batch_size=batch_size, class_mode='categorical', shuffle=True)

    train_gen_DA = train_gen_DAnet(train_gen_source, train_gen_target, batch_size)

    train_stepE = np.floor_divide(train_gen_source.n, batch_size)
    train_stepE_target = np.floor_divide(train_gen_target.n, batch_size)
    #### validation data

    valid_gen_source = datagen.flow_from_directory(source_data + '/validation', target_size=(224, 224), 
                                                   batch_size=batch_size, class_mode='categorical', shuffle=True)
    valid_gen_target = datagen.flow_from_directory(target_data + '/validation', target_size=(224, 224), 
                                                   batch_size=batch_size, class_mode='categorical', shuffle=True)

    #valid_gen_DA = test_gen_DAnet(valid_gen_source, valid_gen_target, batch_size)
    valid_gen_DA = train_gen_DAnet(valid_gen_source, valid_gen_target, batch_size)
    
    valid_stepE = np.floor_divide(valid_gen_source.n, batch_size)
    valid_stepE_target = np.floor_divide(valid_gen_target.n, batch_size)

    #### test data

    test_gen_source = datagen.flow_from_directory(source_data + '/test', target_size=(224, 224), 
                                                   batch_size=batch_size, class_mode='categorical', shuffle=True)
    test_gen_target = datagen.flow_from_directory(target_data + '/test', target_size=(224, 224), 
                                                   batch_size=batch_size, class_mode='categorical', shuffle=True)

    test_gen_DA = test_gen_DAnet(test_gen_source, test_gen_target, batch_size)
    

    test_stepE = np.floor_divide(test_gen_source.n, batch_size)
    test_stepE_target = np.floor_divide(test_gen_target.n, batch_size)   
    
    
    #Define lamFunk
    lamFunk = K.variable(0.0)
    
    for item in MIQ:
        if item=="DA":         
            current_model = Models.DA_model(lamFunk,do_rate_dpl=0, do_rate_lpl=0.5, vgg_train=False, nrUnits=[512,512])
        else:
            current_model = Models.lable_model(do_rate=0.5, vgg_train=False, nrUnits=[512,512])
        
        csv_logger = CSVLogger('C:/Users/DANN_maces_jaskmo/Documents/trainingLog/' + item + 
                        str(now.day) + '-' + str(now.month) + '-' + str(now.year) + '_' + 
                        str(now.hour) + str(now.minute) + '.log')
        if item=='DA':
            current_model.fit_generator(train_gen_DA, train_stepE, epochs=nrEpochs, verbose=1, validation_data=valid_gen_DA, 
                            validation_steps=valid_stepE, callbacks=[csv_logger,FlipControle(lamFunk)], initial_epoch=0,
                            max_queue_size=5, class_weight = {'lplOut':[1,0.33,0.33,0.33,0.5],'dplOut':[1,1]})
        elif item=='source':
            current_model.fit_generator(train_gen_source, train_stepE, epochs=nrEpochs, verbose=1, validation_data=valid_gen_source, 
                            validation_steps=valid_stepE, callbacks=[csv_logger], initial_epoch=0,
                            max_queue_size=5, class_weight = [1,0.33,0.33,0.33,0.5])
        else:
            current_model.fit_generator(train_gen_target, train_stepE_target, epochs=nrEpochs, verbose=1, validation_data=valid_gen_target, 
                            validation_steps=valid_stepE_target, callbacks=[csv_logger], initial_epoch=0,
                            max_queue_size=5, class_weight = [1,0.33,0.33,0.33,0.5])
                     
        if item == "DA":
            DAlpm = utils.dissect_DAlpm(current_model)
            current_model = DAlpm
            
#         #Save the model
#         current_model.save(filepath=path + 'models/'+ item + str(now.day) + '-' + str(now.month) + '-' + str(now.year) + '_' + 
#                        str(now.hour) + str(now.minute) + '.h5')
        
        #TEST
        DTD_dic = {'Physionet':test_gen_source,'Hospital':test_gen_target}
        # save to file 
        test_file = 'C:/Users/DANN_maces_jaskmo/Documents/testLog/'+ item + str(now.day) + '-' + str(now.month) + '-' + str(now.year) + '_' + str(now.hour) + str(now.minute) + '.log'
        sys.stdout = open(test_file, 'w')
        
        for dom in ['Physionet', 'Hospital']:
            test_img, test_lable = DTD_dic[dom].next()
            for count in range(DTD_dic[dom].n//batch_size):
                tmp_img, tmp_lable = DTD_dic[dom].next()
                test_img = np.concatenate((test_img, tmp_img), axis=0)
                test_lable = np.concatenate((test_lable, tmp_lable),axis=0)

            # Compute the test metrecis 
            inv_map = {v: k for k, v in DTD_dic[dom].class_indices.items()}
            target_names = list(inv_map.values())

            targets_test_int = [np.where(r == 1)[0][0] for r in test_lable]
            y_pred = current_model.predict(test_img)
            y_pred2 = np.argmax(y_pred, axis = 1)
            # Test accuracy:
            acc = accuracy_score(targets_test_int, y_pred2)
            
            conf_mat = confusion_matrix(targets_test_int, y_pred2)
            # Per class metrics
            class_report = classification_report(targets_test_int, y_pred2, target_names=target_names)

            print('Accuracy on ' + dom + ' data = ' + str(acc) +'\n \n' + 
                  'Confution matric on ' + dom + ' data: \n' + str(conf_mat) + '\n\n' + 
                  'Class report on ' + dom + ' data: \n' + class_report + '\n\n\n\n')

        sys.stdout = stdout_cell

Found 22053 images belonging to 5 classes.
Found 19037 images belonging to 5 classes.
Found 3485 images belonging to 5 classes.
Found 3675 images belonging to 5 classes.
Found 2816 images belonging to 5 classes.
Found 2404 images belonging to 5 classes.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
lplInput (InputLayer)           (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
model_1 (Model)                 (None, 25088)        14714688    lplInput[0][0]                   
                                                                 dplInput[0][0]                   
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 25088)        

Epoch 1/10
297/297 [==============================] - 76s 254ms/step - loss: 12.1236 - categorical_accuracy: 0.5788 - val_loss: 11.4735 - val_categorical_accuracy: 0.6012
Epoch 2/10
297/297 [==============================] - 75s 251ms/step - loss: 10.7433 - categorical_accuracy: 0.7234 - val_loss: 10.5046 - val_categorical_accuracy: 0.6283
Epoch 3/10
297/297 [==============================] - 75s 252ms/step - loss: 9.8778 - categorical_accuracy: 0.7511 - val_loss: 9.7670 - val_categorical_accuracy: 0.6483
Epoch 4/10
297/297 [==============================] - 74s 251ms/step - loss: 9.1992 - categorical_accuracy: 0.7698 - val_loss: 9.1567 - val_categorical_accuracy: 0.6675
Epoch 5/10
297/297 [==============================] - 74s 251ms/step - loss: 8.6317 - categorical_accuracy: 0.7828 - val_loss: 8.6435 - val_categorical_accuracy: 0.6702
Epoch 6/10
297/297 [==============================] - 75s 251ms/step - loss: 8.1354 - categorical_accuracy: 0.7953 - val_loss: 8.1827 - val_categorical

Epoch 1/10
362/362 [==============================] - 167s 461ms/step - loss: 21.6230 - lplOut_loss: 0.9916 - dplOut_loss: 0.3128 - lplOut_categorical_accuracy: 0.6283 - dplOut_categorical_accuracy: 0.8870 - val_loss: 18.4963 - val_lplOut_loss: 0.5606 - val_dplOut_loss: 0.2025 - val_lplOut_categorical_accuracy: 0.7957 - val_dplOut_categorical_accuracy: 0.9310
0.46211717
Epoch 2/10
362/362 [==============================] - 166s 458ms/step - loss: 16.7827 - lplOut_loss: 0.6472 - dplOut_loss: 0.1883 - lplOut_categorical_accuracy: 0.7753 - dplOut_categorical_accuracy: 0.9312 - val_loss: 15.1272 - val_lplOut_loss: 0.5343 - val_dplOut_loss: 0.1701 - val_lplOut_categorical_accuracy: 0.8136 - val_dplOut_categorical_accuracy: 0.9428
0.7615942
Epoch 3/10
362/362 [==============================] - 166s 460ms/step - loss: 14.0440 - lplOut_loss: 0.5683 - dplOut_loss: 0.1696 - lplOut_categorical_accuracy: 0.7963 - dplOut_categorical_accuracy: 0.9418 - val_loss: 13.0082 - val_lplOut_loss: 0.5153 - v

Epoch 1/10
362/362 [==============================] - 85s 235ms/step - loss: 11.6998 - categorical_accuracy: 0.6674 - val_loss: 10.6250 - val_categorical_accuracy: 0.7969
Epoch 2/10
362/362 [==============================] - 84s 233ms/step - loss: 10.0942 - categorical_accuracy: 0.7727 - val_loss: 9.4879 - val_categorical_accuracy: 0.8163
Epoch 3/10
362/362 [==============================] - 86s 237ms/step - loss: 9.1050 - categorical_accuracy: 0.7944 - val_loss: 8.6633 - val_categorical_accuracy: 0.8216
Epoch 4/10
362/362 [==============================] - 86s 237ms/step - loss: 8.3617 - categorical_accuracy: 0.8081 - val_loss: 8.0153 - val_categorical_accuracy: 0.8300
Epoch 5/10
362/362 [==============================] - 86s 238ms/step - loss: 7.7578 - categorical_accuracy: 0.8240 - val_loss: 7.4844 - val_categorical_accuracy: 0.8331
Epoch 6/10
362/362 [==============================] - 85s 236ms/step - loss: 7.2370 - categorical_accuracy: 0.8334 - val_loss: 7.0077 - val_categorical_

0.9950548
Epoch 7/10
349/349 [==============================] - 159s 457ms/step - loss: 9.6104 - lplOut_loss: 0.4626 - dplOut_loss: 0.4355 - lplOut_categorical_accuracy: 0.8313 - dplOut_categorical_accuracy: 0.8223 - val_loss: 9.3708 - val_lplOut_loss: 0.5051 - val_dplOut_loss: 0.5019 - val_lplOut_categorical_accuracy: 0.8160 - val_dplOut_categorical_accuracy: 0.7764
0.9981779
Epoch 8/10
349/349 [==============================] - 159s 457ms/step - loss: 8.9912 - lplOut_loss: 0.4573 - dplOut_loss: 0.4921 - lplOut_categorical_accuracy: 0.8332 - dplOut_categorical_accuracy: 0.8115 - val_loss: 8.7689 - val_lplOut_loss: 0.5175 - val_dplOut_loss: 0.5226 - val_lplOut_categorical_accuracy: 0.8139 - val_dplOut_categorical_accuracy: 0.7875
0.9993293
Epoch 9/10
349/349 [==============================] - 159s 457ms/step - loss: 8.4211 - lplOut_loss: 0.4395 - dplOut_loss: 0.5464 - lplOut_categorical_accuracy: 0.8402 - dplOut_categorical_accuracy: 0.7536 - val_loss: 8.2145 - val_lplOut_loss: 0.5095 

...

0.9993293
Epoch 9/10
353/353 [==============================] - 161s 457ms/step - loss: 8.4626 - lplOut_loss: 0.4432 - dplOut_loss: 0.5616 - lplOut_categorical_accuracy: 0.8349 - dplOut_categorical_accuracy: 0.7278 - val_loss: 8.4277 - val_lplOut_loss: 0.7792 - val_dplOut_loss: 0.4784 - val_lplOut_categorical_accuracy: 0.7582 - val_dplOut_categorical_accuracy: 0.8324
0.99975324
Epoch 10/10
353/353 [==============================] - 161s 457ms/step - loss: 7.8880 - lplOut_loss: 0.4295 - dplOut_loss: 0.5587 - lplOut_categorical_accuracy: 0.8393 - dplOut_categorical_accuracy: 0.7451 - val_loss: 7.8351 - val_lplOut_loss: 0.6999 - val_dplOut_loss: 0.4998 - val_lplOut_categorical_accuracy: 0.7660 - val_dplOut_categorical_accuracy: 0.8422
0.9999092
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
image_input (InputLayer)     (None, 224, 224, 3)       0         
__________________________________________________

353/353 [==============================] - 84s 237ms/step - loss: 5.5927 - categorical_accuracy: 0.8650 - val_loss: 5.8180 - val_categorical_accuracy: 0.7668
Found 21643 images belonging to 5 classes.
Found 20163 images belonging to 5 classes.
Found 3637 images belonging to 5 classes.
Found 2570 images belonging to 5 classes.
Found 3042 images belonging to 5 classes.
Found 2383 images belonging to 5 classes.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
lplInput (InputLayer)           (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
model_1 (Model)                 (None, 25088)        14714688    lplInput[0][0]                   
                                                                 dplInput[0][0]              

0.9999092
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
image_input (InputLayer)     (None, 224, 224, 3)       0         
_________________________________________________________________
vgg16 (Model)                multiple                  14714688  
_________________________________________________________________
preLp (Flatten)              (None, 25088)             0         
_________________________________________________________________
batch_normalization_4 (Batch (None, 25088)             100352    
_________________________________________________________________
preDnsDo (Dropout)           (None, 25088)             0         
_________________________________________________________________
lpl1 (Dense)                 (None, 512)               12845568  
_________________________________________________________________
batch_normalization_5 (Batch (None, 512)               2048      


Epoch 1/10
345/345 [==============================] - 161s 466ms/step - loss: 21.6148 - lplOut_loss: 0.9100 - dplOut_loss: 0.2782 - lplOut_categorical_accuracy: 0.6572 - dplOut_categorical_accuracy: 0.9043 - val_loss: 18.6768 - val_lplOut_loss: 0.5777 - val_dplOut_loss: 0.1762 - val_lplOut_categorical_accuracy: 0.7904 - val_dplOut_categorical_accuracy: 0.9390
0.46211717
Epoch 2/10
345/345 [==============================] - 159s 461ms/step - loss: 16.9237 - lplOut_loss: 0.6080 - dplOut_loss: 0.1524 - lplOut_categorical_accuracy: 0.7807 - dplOut_categorical_accuracy: 0.9468 - val_loss: 15.4035 - val_lplOut_loss: 0.5474 - val_dplOut_loss: 0.2069 - val_lplOut_categorical_accuracy: 0.7992 - val_dplOut_categorical_accuracy: 0.9207
0.7615942
Epoch 3/10
345/345 [==============================] - 159s 462ms/step - loss: 14.2050 - lplOut_loss: 0.5427 - dplOut_loss: 0.1366 - lplOut_categorical_accuracy: 0.8013 - dplOut_categorical_accuracy: 0.9545 - val_loss: 13.2232 - val_lplOut_loss: 0.5102 - v

Epoch 1/10
345/345 [==============================] - 83s 241ms/step - loss: 11.7855 - categorical_accuracy: 0.6241 - val_loss: 10.6781 - val_categorical_accuracy: 0.7771
Epoch 2/10
345/345 [==============================] - 82s 238ms/step - loss: 10.1122 - categorical_accuracy: 0.7841 - val_loss: 9.5608 - val_categorical_accuracy: 0.7877
Epoch 3/10
345/345 [==============================] - 83s 239ms/step - loss: 9.1178 - categorical_accuracy: 0.8044 - val_loss: 8.7437 - val_categorical_accuracy: 0.7965
Epoch 4/10
345/345 [==============================] - 83s 240ms/step - loss: 8.3581 - categorical_accuracy: 0.8214 - val_loss: 8.0946 - val_categorical_accuracy: 0.7954
Epoch 5/10
345/345 [==============================] - 82s 239ms/step - loss: 7.7426 - categorical_accuracy: 0.8323 - val_loss: 7.5423 - val_categorical_accuracy: 0.7999
Epoch 6/10
345/345 [==============================] - 81s 235ms/step - loss: 7.2191 - categorical_accuracy: 0.8407 - val_loss: 7.0524 - val_categorical_